## Demo of convection in a swollen annulus mesh


NOTE: this notebook can be converted to launch with mpirun

``` bash
jupyter-nbconvert convection_puffed_annulus.ipynb --to python
mpirun python3 convection_puffed_annulus.py
```


In [1]:
import underworld as uw
import glucifer
import numpy as np
from underworld import function as fn
import time

In [2]:
outputDir = "outputCylConv-Ra1e6-32"

In [3]:
# boundary conditions available "BC_FREESLIP, "BC_NOSLIP
bc_wanted = 'BC_FREESLIP'

In [4]:
outer=6.4
inner=3.4

mesh = uw.mesh.FeMesh_PuffedSquareAnnulus(radial_resolution=32, radii=(inner,outer), shadowDepth=2, 
                                    core_stretch=1.5, elementType="Q2/dq1", process_kites=True)

meshT = mesh

radiusFn = mesh.radiusFn
thetaFn  = mesh.thetaFn

unit_radiusFn = mesh.unit_radiusFn
mantleMaskFn  = mesh.shellMaskFn

In [5]:
TField = meshT.add_variable(nodeDofCount=1)
dTField = meshT.add_variable(nodeDofCount=1)

vField = mesh.add_variable(nodeDofCount=mesh.dim)
pField = mesh.subMesh.add_variable(nodeDofCount=1)
pCField = mesh.add_variable(nodeDofCount=1)
ViscField = mesh.add_variable(nodeDofCount=1)

surface       = mesh.specialSets["surface_VertexSet"]
lower_surface = mesh.specialSets["lower_surface_VertexSet"]
excluded      = mesh.specialSets["excluded_VertexSet"]
dead_centre   = mesh.specialSets["dead_centre_VertexSet"]


In [6]:
# create checkpoint function
def checkpoint( mesh, fieldDict, swarm, swarmDict, index, time,
                meshName='mesh', swarmName='swarm', 
                prefix='./', force_mesh=False, enable_xdmf=True):
    import os
    # Check the prefix is valid
    if prefix is not None:
        if not prefix.endswith('/'): prefix += '/' # add a backslash
        if not os.path.exists(prefix) and uw.rank()==0:
            print("Creating directory: {p}".format(p=prefix))
            os.makedirs(prefix)
        uw.barrier() 
            
    if not isinstance(index, int):
        raise TypeError("'index' is not of type int")        
    ii = str(index)
    
    if mesh is not None:
        
        # Error check the mesh and fields
        if not isinstance(mesh, uw.mesh.FeMesh):
            raise TypeError("'mesh' is not of type uw.mesh.FeMesh")
        if not isinstance(fieldDict, dict):
            raise TypeError("'fieldDict' is not of type dict")
        for key, value in fieldDict.items():
            if not isinstance( value, uw.mesh.MeshVariable ):
                raise TypeError("'fieldDict' must contain uw.mesh.MeshVariable elements")


        # see if we have already saved the mesh. It only needs to be saved once
        if not hasattr( checkpoint, 'mH' ):
            checkpoint.mH = mesh.save(prefix+meshName+".h5")
            
        if not force_mesh:
            mh = checkpoint.mH

        for key,value in fieldDict.items():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, mh, meshName, modeltime=time)
        
    # is there a swarm
    if swarm is not None:
        
        # Error check the swarms
        if not isinstance(swarm, uw.swarm.Swarm):
            raise TypeError("'swarm' is not of type uw.swarm.Swarm")
        if not isinstance(swarmDict, dict):
            raise TypeError("'swarmDict' is not of type dict")
        for key, value in swarmDict.items():
            if not isinstance( value, uw.swarm.SwarmVariable ):
                raise TypeError("'fieldDict' must contain uw.swarm.SwarmVariable elements")
    
        sH = swarm.save(prefix+swarmName+"-"+ii+".h5")
        for key,value in swarmDict.i:
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, sH, swarmName, modeltime=time)

## Hidden !

In [7]:
### Boundary conditions

def setVbcs(vField):
    vField.data[...] = 0.
    vField.data[lower_surface.data,:] = (0.0,0.0)
    vField.data[surface.data,:]       = (0.0,0.0)
    vField.data[dead_centre.data,:]   = (0.0,0.0)
    
setVbcs(vField)


if bc_wanted == "BC_NOSLIP":
    # no slip
    
    knockout = surface + lower_surface + dead_centre
    vBC = uw.conditions.CurvilinearDirichletCondition( variable=vField, 
                                                  indexSetsPerDof=(knockout,knockout)
                                                 )

elif bc_wanted == "BC_FREESLIP":
    # free-slip
    
    vBC = uw.conditions.CurvilinearDirichletCondition( variable=vField,
                                                   indexSetsPerDof=(surface+lower_surface,
                                                                    None) )
        
else:
    raise ValueError("Can't find an option for the 'bc_wanted' = {}".format(bc_wanted))
    
    
## Temperature BCs

# Temperature is held constant in the core and the two spherical surfaces

tempBC = uw.conditions.DirichletCondition( variable        = TField, 
                                           indexSetsPerDof = (surface+lower_surface+excluded,) )


def setTBCs(TField):
    TField.data[surface,:] = 0.0
    TField.data[excluded,:] = 1.0      
    TField.data[lower_surface,:] = 1.0 
    return

    

In [8]:
z_hat  = mesh.unitvec_r
unit_radiusFn = mesh.unit_radiusFn

In [9]:

temperatureFn = -0.1 * fn.math.cos(11.0 * thetaFn) * fn.math.sin(np.pi*unit_radiusFn)  + (1.0-unit_radiusFn) 
TField.data[:] = temperatureFn.evaluate(mesh)
setTBCs(TField)


In [10]:
T0 = fn.misc.constant(mesh.mean_value(TField))
T1 = fn.misc.constant(mesh.radial_gradient_value(TField))

Tref = T1 * unit_radiusFn + 1.0 # boundary condition on inner surface
dTFn = (TField - Tref)

Rayleigh_no = 1.0e6
Ra = fn.misc.constant(Rayleigh_no / (outer-inner)**3)

In [11]:
bodyForceFn = Ra * dTFn * z_hat * mantleMaskFn

In [12]:
## Frank-Kamenetskii delta eta = 1000.0 

C0 = fn.misc.constant(1000.0)
C1 = fn.misc.constant(-6.907755)
viscTFn = C0 * fn.math.exp(C1 * TField)

deltaLM = fn.misc.constant(1.0)
viscTFn = fn.misc.constant(1.0)

In [13]:
pCprojector = uw.utils.MeshVariable_Projection(pCField, pField)
pCprojector.solve()

In [14]:
viscosityFn = fn.branching.conditional([ (radiusFn < inner*0.999, 0.001),
                                         (radiusFn < 5.6, deltaLM * viscTFn),
                                         (True, viscTFn) ])

In [15]:
stokesSLE = uw.systems.Curvilinear_Stokes( vField, pField, 
                               fn_viscosity=viscosityFn, 
                               fn_bodyforce=bodyForceFn,
                               conditions=vBC,
                              _removeBCs=False)

In [16]:
advDiff = uw.systems.SLCN_AdvectionDiffusion( phiField=TField, 
                                              velocityField=vField,
                                              fn_diffusivity=1.0,
                                              conditions=tempBC )

In [17]:

# figs = glucifer.Figure(resolution=[1000,1000])
# figs.append( glucifer.objects.Mesh( mesh,  name="Vmesh", segmentsPerEdge=3) )
# figs.append( glucifer.objects.Points(swarm=advDiff._mswarm, fn_size=5.0, fn_colour=1.0, colours="Black"))
# figs.window()


In [18]:
stokesSolver = uw.systems.Solver(stokesSLE)

stokesSolver.set_mg_levels(4)
stokesSolver.set_penalty(10.0)

stokesSolver.options.main.restore_K=True
stokesSolver.options.main.force_correction=True
stokesSolver.options.main.Q22_pc_type="gkgdiag"

stokesSolver.options.A11.ksp_type="fgmres"
stokesSolver.options.scr.ksp_rtol=1.0e-4
stokesSolver.options.scr.ksp_max_it=100
stokesSolver.options.scr.ksp_monitor="ascii"
stokesSolver.options.scr.ksp_type="fgmres"
stokesSolver.options.scr.ksp_sub_iterations=5
stokesSolver.options.scr.ksp_sub_iteration_rtol=3.1

stokesSolver.options.A11.ksp_type="fgmres"
stokesSolver.options.A11.ksp_rtol=1.0e-4
stokesSolver.options.A11.ksp_monitor="ascii"

stokesSolver.options.mg_accel.mg_accelerating_smoothing=True
stokesSolver.options.mg_accel.mg_smooths_to_start=3
stokesSolver.options.mg_accel.mg_smooths_max=12

stokesSolver.options.mg.mg_levels_ksp_convergence_test='skip'
stokesSolver.options.mg.mg_levels_ksp_norm_type='none'
stokesSolver.options.mg.mg_levels_ksp_max_it=5
stokesSolver.options.mg.mg_levels_ksp_type="chebyshev"
stokesSolver.options.mg.mg_coarse_pc_type="lu"
stokesSolver.options.mg.mg_coarse_pc_factor_mat_solver_package="mumps"

# solver.options.mg.mg_coarse_ksp_view="ascii" # to check the above
# stokesSolver.options.mg.mg_levels_ksp_view="ascii"

In [19]:
## Timestepping function

def update_timestep():
    
    
    # Retrieve the maximum possible timestep for the advection-diffusion system.
    
    dt = advDiff.get_max_dt()*2.0
    advDiff.integrate(dt, phiStar=None, interpolator="stripy")
    setTBCs(TField)
    
    T0.value = mesh.mean_value(TField)
    T1.value = mesh.radial_gradient_value(TField)
    
    dTField.data[:] = (dTFn*mantleMaskFn).evaluate(mesh)
        
    # Boundary conditions messed up by rotations
    
    setVbcs(vField)
    stokesSolver.solve(print_stats=False)
 
    # Velocity null space check - very easy while v is in r, theta
    null_space_v  = uw.utils.Integral(vField[1] * radiusFn * mantleMaskFn, mesh).evaluate()[0] 
    null_space_v /= uw.utils.Integral( (radiusFn * mantleMaskFn)**2, mesh).evaluate()[0] 
    # print("Null Space Velocity: {}".format(null_space_v))
         
    ## Back to xyz 
    uw.libUnderworld.Underworld.AXequalsX( stokesSLE._rot._cself, stokesSLE._velocitySol._cself, False)

    # Clean up the solution
    vField.data[:,:] -= null_space_v * (mesh.unitvec_theta * radiusFn * mantleMaskFn).evaluate(mesh)[:,:]
    vField.data[excluded.data,:] = 0.0

    pCprojector.solve()    
    
        
    return time+dt, step+1, 

In [20]:
# init 
time = 0.
step = 0
steps_end = 5000

import pathlib

vdotv = fn.math.dot(vField,vField)

pathlib.Path(outputDir).mkdir(parents=True, exist_ok=True)

store = glucifer.Store(outputDir+'/VizData')
timelog = outputDir+"/timelog.txt"
timelogF = open(timelog, 'w')
print("#{:>8s} |{:>15s} |{:>15s} |{:>15s} |{:>15s}  ".format("Step", "Time", "Vrms", "Nu", "NullV"), 
          file=timelogF,
              flush=True) 


fig = glucifer.Figure(store=store)
fig.append( glucifer.objects.Mesh( mesh,  name="Vmesh") )
fig.append( glucifer.objects.Surface(mesh=mesh, onMesh=True, fn=pCField, name="Pressure") )
fig.append( glucifer.objects.Surface(mesh=mesh, fn=dTFn*mantleMaskFn, onMesh=True, name="dTemp") )
fig.append( glucifer.objects.Surface(mesh=mesh, fn=vdotv, onMesh=True, name="Vmag") )
fig.append( glucifer.objects.Surface(mesh=mesh, fn=TField, onMesh=True, name="Temp") )


In [21]:

checkpoint_fieldDict = {'velocity':vField,
                        'pressure':pCField,
                        'temperature':TField,
                        # 'viscosity':ViscField 
                       }

setVbcs(vField)
stokesSolver.solve(print_stats=True)
uw.libUnderworld.Underworld.AXequalsX( stokesSLE._rot._cself, stokesSLE._velocitySol._cself, False)
vField.data[excluded.data,:] = 0.0

store.step = step
fig.save()
checkpoint( mesh, checkpoint_fieldDict, step, time,
            prefix=outputDir, force_mesh=False, enable_xdmf=True)
 


# perform timestepping
while step < steps_end:
    uw.timing.start()


    # Solve for the velocity field given the current temperature field.
    time, step = update_timestep()
    
    vrms = np.sqrt( mesh.integrate(vdotv)[0] / mesh.integrate(1.)[0] )
    HFlux = fn.math.dot(TField.fn_gradient, mesh.unitvec_r)
    Nu = -1.0 * uw.utils.Integral(mesh=mesh, fn=HFlux, integrationType='surface', surfaceIndexSet=surface).evaluate()[0]
    Nu /= uw.utils.Integral(mesh=mesh, fn=1.0, integrationType='surface', surfaceIndexSet=surface).evaluate()[0]
    Nu *= (outer-inner)
 
    if uw.rank() == 0:
        print("Step {s:05d}".format(s=step))
        print(" {s:8d} |{t:15.5e} |{vrms:15.5e} |{nu:15.5e} |{nulv:15.5e}  ".format(s=step, 
                                                                      t=time, 
                                                                      vrms=vrms,
                                                                      nu=Nu,
                                                                      nulv=null_space_v), 
              file=timelogF,
              flush=True)

    # Evaluate the viscosity for store
    ViscField.data[:] = viscosityFn.evaluate(mesh)
    
    if step%10==0:
        store.step = step
        fig.save()
        
    if step%100==0:
        checkpoint( mesh, checkpoint_fieldDict, step, time,
                    prefix=outputDir, force_mesh=False, enable_xdmf=True)
        
    
    uw.timing.print_table()
    


      

Setup - BCs        1.907e-05 s
Setup - Eq numbers 0.000102 s
Setup - Zero vecs  3.099e-05 s
Setup - Matrices   1.855 s
Setup - Vectors    1.228 s

 
Pressure iterations:   6
Velocity iterations:  37 (presolve)      
Velocity iterations: 311 (pressure solve)
Velocity iterations:   5 (backsolve)     
Velocity iterations: 353 (total solve)   
 
SCR RHS  setup time: 3.2031e-02
SCR RHS  solve time: 1.0403e+00
Pressure setup time: 5.5401e-03
Pressure solve time: 6.6602e+00
Velocity setup time: 9.3448e-03 (backsolve)
Velocity solve time: 1.2855e-01 (backsolve)
Total solve time   : 8.0410e+00
 



TypeError: checkpoint() missing 2 required positional arguments: 'index' and 'time'

In [ ]:
# fig.window()

In [ ]:

%%sh
rm outputCylConvTest/VizData.gldb